In [27]:
%matplotlib inline
import matplotlib.pyplot as plt
import torch
from encoder import Encoder
import k2
import sentencepiece as spm
device = "cuda:0"

In [24]:
encoder = Encoder(num_features=80, channels=256, channels_expansion=1024, dilations=[1 for _ in range(11)], kernel_size=8,
                  activation='ReLU', se_activation='ReLU', scaled_conv=True, act_bal=True, se_gate='tanh', zero_init_residual=True,
                  gamma=0.93, chunksize=16, output_channels=512)
checkpoint = torch.load("korean/vocab500_ipa3_noemawd_l11_customfbank_adamw/epoch-10.pt")
new_checkpoint = dict()
for name, param in checkpoint["model"].items():
    if name.startswith("encoder"):
        new_checkpoint[name[len("encoder."):]] = param
encoder.load_state_dict(new_checkpoint)
encoder.to(device)
batch = torch.load("korean/vocab500_ipa3_noemawd_l11_customfbank_adamw/batch-8d62d777-8090-c4bd-d6a7-4dbe3e572e0f.pt")

In [ ]:
sp = spm.SentencePieceProcessor()
sp.load("/home/shahn/Documents/icefall_github/egs/ksponspeech/ASR/data/lang_bpe_500_ipa_max3/bpe.model")

In [25]:
y = sp.encode(batch["supervisions"]["custom.ipa_filtered"], out_type=int)
y = k2.RaggedTensor(y).to(device)
with torch.cuda.amp.autocast():
    out = encoder(batch["inputs"].to(device), batch["supervisions"]["num_frames"].to(device))

In [26]:
out

(tensor([[[-6.3184e-01, -1.9910e-01,  1.0797e-01,  ..., -2.8589e-01,
            4.8291e-01, -3.3081e-01],
          [-3.7402e-01,  6.8176e-02, -3.2812e-01,  ..., -3.1299e-01,
            6.4160e-01, -1.9507e-01],
          [ 1.3025e-01, -3.6743e-01, -5.5859e-01,  ..., -3.8818e-01,
            1.3242e+00,  4.3042e-01],
          ...,
          [-1.0889e+00,  2.7237e-02,  9.5703e-01,  ..., -6.4844e-01,
            5.8398e-01,  8.5596e-01],
          [-1.0996e+00, -4.3628e-01,  8.0811e-01,  ..., -8.4863e-01,
            1.6104e+00,  8.7012e-01],
          [-1.1768e+00,  8.0261e-02,  7.7881e-01,  ..., -1.1445e+00,
            1.9033e+00,  1.1230e+00]],
 
         [[ 9.6289e-01, -5.4980e-01,  1.9470e-01,  ...,  4.0723e-01,
            1.0479e+00, -1.6123e+00],
          [ 1.2812e+00, -3.6713e-02, -3.9087e-01,  ...,  3.1250e-02,
            1.0127e+00, -6.0596e-01],
          [ 7.6465e-01, -1.0850e+00, -5.8014e-02,  ...,  4.2554e-01,
            1.3516e+00,  3.2446e-01],
          ...,
    